In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import log_loss

from crepes import ConformalPredictiveSystem
from crepes.extras import binning, DifficultyEstimator

from calibrated_explanations import  VennAbers, __version__
from calibrated_explanations.utils import transform_to_numeric
from time import time

import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')

print(f"calibrated_explanations {__version__}")



calibrated_explanations v0.3.1


In [4]:

datasets = {'COMPAS': 'classification', 'Adult': 'classification', 'German': 'classification', 'Housing': 'regression', 'CaC': 'regression'}
results = {}
for name, type in datasets.items():
    results[name] = {}
    results[name]['type'] = type
    for category in ['mondrian', 'mondrian_x', 'model_split']:
        results[name][category] = {'time': 0, 'base': 0, 'rand': 0, 'race': 0, 'sex': 0, 'age': 0, 'edu': 0, 'crm': 0}

num_bins = 5
num_rep = 1
kf = KFold(n_splits=10)

In [5]:
def coverage(y_true, low, up):
    return np.mean((y_true >= low) & (y_true <= up))

# COMPAS
Data preprocessing from [this notebook](https://colab.research.google.com/github/pair-code/what-if-tool/blob/master/WIT_COMPAS_with_SHAP.ipynb#scrollTo=KF00pJvkeicT).

In [255]:
dataset = 'COMPAS'
df = pd.read_csv('https://storage.googleapis.com/what-if-tool-resources/computefest2019/cox-violent-parsed_filt.csv')
print(df.shape)
# Preprocess the data

# Filter out entries with no indication of recidivism or no compass score
df = df[df['is_recid'] != -1]
df = df[df['decile_score'] != -1]


# Make the COMPASS label column numeric (0 and 1), for use in our model
df['score_text'] = np.where(df['score_text'] == 'Low', 'Low', 'Not Low')

target = 'score_text'

(18316, 40)


In [256]:
features_to_keep = ['sex', 'age', 'race', 'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count', 'score_text']
# 'age_cat', 'is_recid', 'vr_charge_desc',  'c_charge_desc',, 'is_violent_recid''vr_charge_degree', 'c_charge_degree', 'decile_score', 
df = df[features_to_keep]
df, categorical_features, categorical_labels, target_labels, mappings = transform_to_numeric(df, target)


In [257]:
print(df.columns)
race = 2
age = 1
sex = 0

Index(['sex', 'age', 'race', 'juv_fel_count', 'juv_misd_count',
       'juv_other_count', 'priors_count', 'score_text'],
      dtype='object')


In [258]:
num_to_test = 20 # number of instances to test

df = df.sample(frac=1, random_state=42).sort_values(by=[target])
Xd, yd = df.drop([target],axis=1), df[target] 
no_of_classes = len(np.unique(yd))
no_of_features = Xd.shape[1]
no_of_instances = Xd.shape[0]

# select test instances from each class and split into train, cal and test

X, y = Xd.values, yd.values

In [259]:
# Mondrian with feature exclusion

base = 'base'
rand = 'rand'
attributes = {base:base, rand:rand, 'race':race, 'sex':sex, 'age':age}
attribute_names = [base, rand, 'race', 'sex', 'age']
va = {}
mondrian_x = {}
mondrian_x['uncal'] = {}
mondrian_x['proba'] = {}
mondrian_x['low'] = {}
mondrian_x['high'] = {}
mondrian_x['width'] = {}
mondrian_x['y'] = []
time_mondrian_x = time()
for attr in attribute_names:
    va[attr] = None
    mondrian_x['uncal'][attr] = []
    mondrian_x['proba'][attr] = []
    mondrian_x['low'][attr] = []
    mondrian_x['high'][attr] = []
    mondrian_x['width'][attr] = []

for l in range(num_rep):
    np.random.seed(l)
    indices = np.random.permutation(no_of_instances)
    X = X[indices,:]
    y = y[indices]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        x_train, x_cal, x_test = X_train, X_cal, X_test        
        for attr, a in attributes.items():
            if attr not in [base, rand]:
                x_train = np.delete(x_train, a, axis=1)
                x_cal = np.delete(x_cal, a, axis=1)
                x_test = np.delete(x_test, a, axis=1)
        model = RandomForestClassifier()
        model.fit(x_train,y_train)
        
        age_cal_bin, age_cal_boundaries = binning(X_cal[:,age], bins=num_bins)
        age_test_bin = binning(X_test[:,age], age_cal_boundaries)
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=num_bins)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        cal_bins = {base: None, rand: rand_cal_bin, 'race': X_cal[:,race], 'sex': X_cal[:,sex], 'age': age_cal_bin}
        test_bins = {base: None, rand: rand_test_bin, 'race': X_test[:,race], 'sex': X_test[:,sex], 'age': age_test_bin}
        
        mondrian_x['y'].append(y_test)
        
        for attr, a in attributes.items():                
            uncal = model.predict_proba(x_test)
            va[attr] = VennAbers(model.predict_proba(x_cal), y_cal, model, bins=cal_bins[attr])
            proba, low, high = va[attr].predict_proba(x_test, output_interval=True, bins=test_bins[attr])
            mondrian_x['uncal'][attr].append(uncal[:,1])
            mondrian_x['proba'][attr].append(proba[:,1])
            mondrian_x['low'][attr].append(low)
            mondrian_x['high'][attr].append(high)
            mondrian_x['width'][attr].append(high-low)

results[dataset]['mondrian_x']['time'] = time() - time_mondrian_x
results[dataset]['mondrian_x']['struct'] = mondrian_x

In [260]:
# Mondrian with all features

base = 'base'
rand = 'rand'
attributes = [base, rand, race, sex, age]
attribute_names = [base, rand, 'race', 'sex', 'age']
va = {}
mondrian = {}
mondrian['uncal'] = {}
mondrian['proba'] = {}
mondrian['low'] = {}
mondrian['high'] = {}
mondrian['width'] = {}
mondrian['y'] = []
time_mondrian = time()
for attr in attribute_names:
    va[attr] = None
    mondrian['uncal'][attr] = []
    mondrian['proba'][attr] = []
    mondrian['low'][attr] = []
    mondrian['high'][attr] = []
    mondrian['width'][attr] = []

for l in range(num_rep):
    np.random.seed(l)
    indices = np.random.permutation(no_of_instances)
    X = X[indices,:]
    y = y[indices]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        
        model = RandomForestClassifier()

        model.fit(X_train,y_train)
        age_cal_bin, age_cal_boundaries = binning(X_cal[:,age], bins=num_bins)
        age_test_bin = binning(X_test[:,age], age_cal_boundaries)
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=num_bins)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        cal_bins = {base: None, rand: rand_cal_bin, 'race': X_cal[:,race], 'sex': X_cal[:,sex], 'age': age_cal_bin}
        test_bins = {base: None, rand: rand_test_bin, 'race': X_test[:,race], 'sex': X_test[:,sex], 'age': age_test_bin}
        
        mondrian['y'].append(y_test)
        
        for attr in attribute_names:
            uncal = model.predict_proba(X_test)
            va[attr] = VennAbers(model.predict_proba(X_cal), y_cal, model, bins=cal_bins[attr])
            proba, low, high = va[attr].predict_proba(X_test, output_interval=True, bins=test_bins[attr])
            mondrian['uncal'][attr].append(uncal[:,1])
            mondrian['proba'][attr].append(proba[:,1])
            mondrian['low'][attr].append(low)
            mondrian['high'][attr].append(high)
            mondrian['width'][attr].append(high-low)

results[dataset]['mondrian']['time'] = time() - time_mondrian
results[dataset]['mondrian']['struct'] = mondrian

In [261]:
# Model split based on Mondrian categories

base = 'base'
rand = 'rand'
attributes = [base, rand, race, sex, age]
attribute_names = [base, rand, 'race', 'sex', 'age']
model_split = {}
model_split['uncal'] = {}
model_split['proba'] = {}
model_split['low'] = {}
model_split['high'] = {}
model_split['width'] = {}
model_split['y'] = {}
for attr in attribute_names:
    model_split['uncal'][attr] = []
    model_split['proba'][attr] = []
    model_split['low'][attr] = []
    model_split['high'][attr] = []
    model_split['width'][attr] = []
    model_split['y'][attr] = []
    
time_split = time()
for l in range(num_rep):
    np.random.seed(l)
    indices = np.random.permutation(no_of_instances)
    X = X[indices,:]
    y = y[indices]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        
        age_cal_bin, age_cal_boundaries = binning(X_cal[:,age], bins=num_bins)
        age_train_bin = binning(X_train[:,age], age_cal_boundaries)
        age_test_bin = binning(X_test[:,age], age_cal_boundaries)
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=num_bins)
        rand_train_bin = binning(np.random.rand(len(y_train)), rand_cal_boundaries)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        train_bins = {base: np.ones(len(y_train)), rand: rand_train_bin, 'race': X_train[:,race], 'sex': X_train[:,sex], 'age': age_train_bin}
        cal_bins = {base: np.ones(len(y_cal)), rand: rand_cal_bin, 'race': X_cal[:,race], 'sex': X_cal[:,sex], 'age': age_cal_bin}
        test_bins = {base: np.ones(len(y_test)), rand: rand_test_bin, 'race': X_test[:,race], 'sex': X_test[:,sex], 'age': age_test_bin}
        
        
        for attr in attribute_names:
            for bin in np.unique(train_bins[attr]):
                X_train_bin = X_train[train_bins[attr] == bin]
                y_train_bin = y_train[train_bins[attr] == bin]
                X_cal_bin = X_cal[cal_bins[attr] == bin]
                y_cal_bin = y_cal[cal_bins[attr] == bin]
                X_test_bin = X_test[test_bins[attr] == bin]
                y_test_bin = y_test[test_bins[attr] == bin]
                
                if len(y_train_bin) == 0 or len(y_cal_bin) == 0 or len(y_test_bin) == 0:
                    continue
                
                model = RandomForestClassifier()                
                model.fit(X_train_bin,y_train_bin)
                uncal = model.predict_proba(X_test_bin)
                va = VennAbers(model.predict_proba(X_cal_bin), y_cal_bin, model)
                proba, low, high = va.predict_proba(X_test_bin, output_interval=True)
                model_split['uncal'][attr].append(uncal[:,1])
                model_split['proba'][attr].append(proba[:,1])
                model_split['low'][attr].append(low)
                model_split['high'][attr].append(high)
                model_split['width'][attr].append(high - low)
                model_split['y'][attr].append(y_test_bin)

results[dataset]['model_split']['time'] = time() - time_split
results[dataset]['model_split']['struct'] = model_split

In [262]:
for attr in attribute_names:
    mondrian_x['uncal'][attr] = np.concatenate(mondrian_x['uncal'][attr])
    mondrian_x['proba'][attr] = np.concatenate(mondrian_x['proba'][attr]) 
    mondrian_x['low'][attr] = np.concatenate(mondrian_x['low'][attr]) 
    mondrian_x['high'][attr] = np.concatenate(mondrian_x['high'][attr]) 
    mondrian_x['width'][attr] = np.concatenate(mondrian_x['width'][attr]) 
    mondrian['uncal'][attr] = np.concatenate(mondrian['uncal'][attr])
    mondrian['proba'][attr] = np.concatenate(mondrian['proba'][attr]) 
    mondrian['low'][attr] = np.concatenate(mondrian['low'][attr]) 
    mondrian['high'][attr] = np.concatenate(mondrian['high'][attr]) 
    mondrian['width'][attr] = np.concatenate(mondrian['width'][attr]) 
    model_split['uncal'][attr] = np.concatenate(model_split['uncal'][attr])
    model_split['proba'][attr] = np.concatenate(model_split['proba'][attr]) 
    model_split['low'][attr] = np.concatenate(model_split['low'][attr]) 
    model_split['high'][attr] = np.concatenate(model_split['high'][attr]) 
    model_split['width'][attr] = np.concatenate(model_split['width'][attr]) 
    model_split['y'][attr] = np.concatenate(model_split['y'][attr]) 

mondrian_x['y'] = np.concatenate(mondrian_x['y']) 
mondrian['y'] = np.concatenate(mondrian['y']) 

In [263]:
print('Category\t CondIncl\t CondExcl\t ModelSplit')
for attr in attribute_names:
    results[dataset]['mondrian_x'][attr] = np.mean(mondrian_x['width'][attr])
    results[dataset]['mondrian'][attr] = np.mean(mondrian['width'][attr])
    results[dataset]['model_split'][attr] = np.mean(model_split['width'][attr])
    print(f'{attr} width: \t{np.mean(mondrian["width"][attr]): .3f} \t{np.mean(mondrian_x["width"][attr]): .3f} \t{np.mean(model_split["width"][attr]): .3f}')

Category	 CondIncl	 CondExcl	 ModelSplit
base width: 	 0.0075 	 0.0082 	 0.0074
rand width: 	 0.0226 	 0.0234 	 0.0205
race width: 	 0.0184 	 0.0198 	 0.0177
sex width: 	 0.0118 	 0.0126 	 0.0111
age width: 	 0.0218 	 0.0250 	 0.0214


In [264]:
print('\t CondIncl\t CondExcl\t ModelSplit\t CondIncl\t CondExcl\t ModelSplit')
print('Category\t VA\t VA\t VA\t Uncal\t Uncal\t Uncal')
for attr in attribute_names:
    print(f'{attr} Accuracy: \t{np.mean((mondrian["proba"][attr] > 0.5) == (mondrian["y"] == 1)): .3f} \t{np.mean((mondrian_x["proba"][attr] > 0.5) == (mondrian_x["y"] == 1)): .3f} \t{np.mean((model_split["proba"][attr] > 0.5) == (model_split["y"][attr] == 1)): .3f}', end='')
    print(f' \t{np.mean((mondrian["uncal"][attr] > 0.5) == (mondrian["y"] == 1)): .3f} \t{np.mean((mondrian_x["uncal"][attr] > 0.5) == (mondrian_x["y"] == 1)): .3f} \t{np.mean((model_split["uncal"][attr] > 0.5) == (model_split["y"][attr] == 1)): .3f}')
print()
for attr in attribute_names:
    print(f'{attr} LogLoss: \t{log_loss(mondrian["y"], mondrian["proba"][attr]): .3f} \t{log_loss(mondrian_x["y"], mondrian_x["proba"][attr]): .3f} \t{log_loss(model_split["y"][attr], model_split["proba"][attr]): .3f}', end='')
    print(f' \t{log_loss(mondrian["y"], mondrian["uncal"][attr]): .3f} \t{log_loss(mondrian_x["y"], mondrian_x["uncal"][attr]): .3f} \t{log_loss(model_split["y"][attr], model_split["uncal"][attr]): .3f}')
    

	 CondIncl	 CondExcl	 ModelSplit	 CondIncl	 CondExcl	 ModelSplit
Category	 VA	 VA	 VA	 Uncal	 Uncal	 Uncal
base Accuracy: 	 0.7783 	 0.7514 	 0.7753 	 0.7769 	 0.7528 	 0.7743
rand Accuracy: 	 0.7743 	 0.7489 	 0.7318 	 0.7769 	 0.7528 	 0.7339
race Accuracy: 	 0.7765 	 0.7542 	 0.7736 	 0.7769 	 0.7528 	 0.7736
sex Accuracy: 	 0.7785 	 0.7528 	 0.7742 	 0.7769 	 0.7528 	 0.7724
age Accuracy: 	 0.7770 	 0.7476 	 0.7712 	 0.7769 	 0.7528 	 0.7723

base LogLoss: 	 0.4612 	 0.4973 	 0.4644 	 0.7691 	 0.6797 	 0.8008
rand LogLoss: 	 0.4660 	 0.5009 	 0.5367 	 0.7691 	 0.6797 	 0.9092
race LogLoss: 	 0.4608 	 0.4865 	 0.4648 	 0.7691 	 0.6797 	 0.8268
sex LogLoss: 	 0.4618 	 0.4962 	 0.4653 	 0.7691 	 0.6797 	 0.7937
age LogLoss: 	 0.4611 	 0.4957 	 0.4688 	 0.7691 	 0.6797 	 0.8015


In [265]:
import pickle

with open(f'../evaluation/results_mondrian_{dataset}_{str(num_rep)}.pkl', 'wb') as file:
    pickle.dump(results, file)


# Adult

In [266]:
dataset = 'Adult'
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
adult = fetch_ucirepo(id=2) 
  
# data (as pandas dataframes) 
X = adult.data.features 
y = adult.data.targets 

In [267]:
df = X
target = 'Income'
y = y.replace('<=50K.', '<=50K')
y = y.replace('>50K.', '>50K')
df[target] = y
df = df.dropna()
df, categorical_features, categorical_labels, target_labels, _ = transform_to_numeric(df, target)
print(target_labels)

{0: '<=50K', 1: '>50K'}


In [268]:
num_to_test = 10 # number of instances to test, one from each class

df = df.sample(frac=1, random_state=42).sort_values(by=[target])
Xd, yd = df.drop(target,axis=1), df[target] 
no_of_classes = len(np.unique(yd))
no_of_features = Xd.shape[1]
no_of_instances = Xd.shape[0]

# select test instances from each class and split into train, cal and test
X, y = Xd.values, yd.values

In [269]:
print(df.columns)
edu = 3
race = 8
sex = 9

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'Income'],
      dtype='object')


In [270]:
# Mondrian with feature exclusion

base = 'base'
rand = 'rand'
attributes = {base:base, rand:rand, 'race':race, 'sex':sex, 'edu':edu}
attribute_names = [base, rand, 'race', 'sex', 'edu']
va = {}
mondrian_x = {}
mondrian_x['uncal'] = {}
mondrian_x['proba'] = {}
mondrian_x['low'] = {}
mondrian_x['high'] = {}
mondrian_x['width'] = {}
mondrian_x['y'] = []
time_mondrian_x = time()
for attr in attribute_names:
    va[attr] = None
    mondrian_x['uncal'][attr] = []
    mondrian_x['proba'][attr] = []
    mondrian_x['low'][attr] = []
    mondrian_x['high'][attr] = []
    mondrian_x['width'][attr] = []

for l in range(num_rep):
    np.random.seed(l)
    indices = np.random.permutation(no_of_instances)
    X = X[indices,:]
    y = y[indices]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        x_train, x_cal, x_test = X_train, X_cal, X_test        
        for attr, a in attributes.items():
            if attr not in [base, rand]:
                x_train = np.delete(x_train, a, axis=1)
                x_cal = np.delete(x_cal, a, axis=1)
                x_test = np.delete(x_test, a, axis=1)
        model = RandomForestClassifier()
        model.fit(x_train,y_train)
        
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=num_bins)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        cal_bins = {base: None, rand: rand_cal_bin, 'race': X_cal[:,race], 'sex': X_cal[:,sex], 'edu': X_cal[:,edu]}
        test_bins = {base: None, rand: rand_test_bin, 'race': X_test[:,race], 'sex': X_test[:,sex], 'edu': X_test[:,edu]}
        
        mondrian_x['y'].append(y_test)
        
        for attr, a in attributes.items():            
            uncal = model.predict_proba(x_test)
            va[attr] = VennAbers(model.predict_proba(x_cal), y_cal, model, bins=cal_bins[attr])
            proba, low, high = va[attr].predict_proba(x_test, output_interval=True, bins=test_bins[attr])
            mondrian_x['uncal'][attr].append(uncal[:,1])
            mondrian_x['proba'][attr].append(proba[:,1])
            mondrian_x['low'][attr].append(low)
            mondrian_x['high'][attr].append(high)
            mondrian_x['width'][attr].append(high-low)

results[dataset]['mondrian_x']['time'] = time() - time_mondrian_x
results[dataset]['mondrian_x']['struct'] = mondrian_x

In [271]:
# Mondrian with all features

base = 'base'
rand = 'rand'
attributes = [base, rand, race, sex, edu]
attribute_names = [base, rand, 'race', 'sex', 'edu']
va = {}
mondrian = {}
mondrian['uncal'] = {}
mondrian['proba'] = {}
mondrian['low'] = {}
mondrian['high'] = {}
mondrian['width'] = {}
mondrian['y'] = []
time_mondrian = time()
for attr in attribute_names:
    va[attr] = None
    mondrian['uncal'][attr] = []
    mondrian['proba'][attr] = []
    mondrian['low'][attr] = []
    mondrian['high'][attr] = []
    mondrian['width'][attr] = []

time_mondrian = time()
for l in range(num_rep):
    np.random.seed(l)
    indices = np.random.permutation(no_of_instances)
    X = X[indices,:]
    y = y[indices]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        
        model = RandomForestClassifier()
        model.fit(X_train,y_train)
        
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=num_bins)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        cal_bins = {base: None, rand: rand_cal_bin, 'race': X_cal[:,race], 'sex': X_cal[:,sex], 'edu': X_cal[:,edu]}
        test_bins = {base: None, rand: rand_test_bin, 'race': X_test[:,race], 'sex': X_test[:,sex], 'edu': X_test[:,edu]}
        
        mondrian['y'].append(y_test)
        
        for attr in attribute_names:
            uncal = model.predict_proba(X_test)
            va[attr] = VennAbers(model.predict_proba(X_cal), y_cal, model, bins=cal_bins[attr])
            proba, low, high = va[attr].predict_proba(X_test, output_interval=True, bins=test_bins[attr])
            mondrian['uncal'][attr].append(uncal[:,1])
            mondrian['proba'][attr].append(proba[:,1])
            mondrian['low'][attr].append(low)
            mondrian['high'][attr].append(high)
            mondrian['width'][attr].append(high-low)

results[dataset]['mondrian']['time'] = time() - time_mondrian
results[dataset]['mondrian']['struct'] = mondrian

In [272]:
# Model split based on Mondrian categories

base = 'base'
rand = 'rand'
attributes = [base, rand, race, sex, edu]
attribute_names = [base, rand, 'race', 'sex', 'edu']
model_split = {}
model_split['uncal'] = {}
model_split['proba'] = {}
model_split['low'] = {}
model_split['high'] = {}
model_split['width'] = {}
model_split['y'] = {}
for attr in attribute_names:
    model_split['uncal'][attr] = []
    model_split['proba'][attr] = []
    model_split['low'][attr] = []
    model_split['high'][attr] = []
    model_split['width'][attr] = []
    model_split['y'][attr] = []

time_split = time()
for l in range(num_rep):
    np.random.seed(l)
    indices = np.random.permutation(no_of_instances)
    X = X[indices,:]
    y = y[indices]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        

        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=num_bins)
        rand_train_bin = binning(np.random.rand(len(y_train)), rand_cal_boundaries)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        train_bins = {base: np.ones(len(y_train)), rand: rand_train_bin, 'race': X_train[:,race], 'sex': X_train[:,sex], 'edu': X_train[:,edu]}
        cal_bins = {base: np.ones(len(y_cal)), rand: rand_cal_bin, 'race': X_cal[:,race], 'sex': X_cal[:,sex], 'edu': X_cal[:,edu]}
        test_bins = {base: np.ones(len(y_test)), rand: rand_test_bin, 'race': X_test[:,race], 'sex': X_test[:,sex], 'edu': X_test[:,edu]}
        
        try:
            for attr in attribute_names:
                for bin in np.unique(train_bins[attr]):
                    X_train_bin = X_train[train_bins[attr] == bin]
                    y_train_bin = y_train[train_bins[attr] == bin]
                    X_cal_bin = X_cal[cal_bins[attr] == bin]
                    y_cal_bin = y_cal[cal_bins[attr] == bin]
                    X_test_bin = X_test[test_bins[attr] == bin]
                    y_test_bin = y_test[test_bins[attr] == bin]
                
                    if len(y_train_bin) == 0 or len(y_cal_bin) == 0 or len(y_test_bin) == 0:
                        continue
                    
                    model = RandomForestClassifier()
                    model.fit(X_train_bin,y_train_bin)
                    uncal = model.predict_proba(X_test_bin)
                    va = VennAbers(model.predict_proba(X_cal_bin), y_cal_bin, model)
                    proba, low, high = va.predict_proba(X_test_bin, output_interval=True)
                    model_split['uncal'][attr].append(uncal[:,1])
                    model_split['proba'][attr].append(proba[:,1])
                    model_split['low'][attr].append(low)
                    model_split['high'][attr].append(high)
                    model_split['width'][attr].append(high - low)
                    model_split['y'][attr].append(y_test_bin)
        except Exception as e:
            print(e)
            print(f'attr: {attr} bin: {bin}')    

results[dataset]['model_split']['time'] = time() - time_split
results[dataset]['model_split']['struct'] = model_split

index 1 is out of bounds for axis 1 with size 1
attr: edu bin: 14


In [273]:
for attr in attribute_names:
    mondrian_x['uncal'][attr] = np.concatenate(mondrian_x['uncal'][attr])
    mondrian_x['proba'][attr] = np.concatenate(mondrian_x['proba'][attr]) 
    mondrian_x['low'][attr] = np.concatenate(mondrian_x['low'][attr]) 
    mondrian_x['high'][attr] = np.concatenate(mondrian_x['high'][attr]) 
    mondrian_x['width'][attr] = np.concatenate(mondrian_x['width'][attr]) 
    mondrian['uncal'][attr] = np.concatenate(mondrian['uncal'][attr])
    mondrian['proba'][attr] = np.concatenate(mondrian['proba'][attr]) 
    mondrian['low'][attr] = np.concatenate(mondrian['low'][attr]) 
    mondrian['high'][attr] = np.concatenate(mondrian['high'][attr]) 
    mondrian['width'][attr] = np.concatenate(mondrian['width'][attr]) 
    model_split['uncal'][attr] = np.concatenate(model_split['uncal'][attr])
    model_split['proba'][attr] = np.concatenate(model_split['proba'][attr]) 
    model_split['low'][attr] = np.concatenate(model_split['low'][attr]) 
    model_split['high'][attr] = np.concatenate(model_split['high'][attr]) 
    model_split['width'][attr] = np.concatenate(model_split['width'][attr]) 
    model_split['y'][attr] = np.concatenate(model_split['y'][attr]) 

mondrian_x['y'] = np.concatenate(mondrian_x['y']) 
mondrian['y'] = np.concatenate(mondrian['y']) 

In [274]:
print('Category\t CondIncl\t CondExcl\t ModelSplit')
for attr in attribute_names:
    results[dataset]['mondrian_x'][attr] = np.mean(mondrian_x['width'][attr])
    results[dataset]['mondrian'][attr] = np.mean(mondrian['width'][attr])
    results[dataset]['model_split'][attr] = np.mean(model_split['width'][attr])
    print(f'{attr} width: \t{np.mean(mondrian["width"][attr]): .3f} \t{np.mean(mondrian_x["width"][attr]): .3f} \t{np.mean(model_split["width"][attr]): .3f}')

Category	 CondIncl	 CondExcl	 ModelSplit
base width: 	 0.0029 	 0.0029 	 0.0030
rand width: 	 0.0098 	 0.0095 	 0.0099
race width: 	 0.0068 	 0.0067 	 0.0069
sex width: 	 0.0046 	 0.0045 	 0.0048
edu width: 	 0.0161 	 0.0147 	 0.0152


In [275]:
print('\t CondIncl\t CondExcl\t ModelSplit\t CondIncl\t CondExcl\t ModelSplit')
print('Category\t VA\t VA\t VA\t Uncal\t Uncal\t Uncal')
for attr in attribute_names:
    print(f'{attr} Accuracy: \t{np.mean((mondrian["proba"][attr] > 0.5) == (mondrian["y"] == 1)): .3f} \t{np.mean((mondrian_x["proba"][attr] > 0.5) == (mondrian_x["y"] == 1)): .3f} \t{np.mean((model_split["proba"][attr] > 0.5) == (model_split["y"][attr] == 1)): .3f}', end='')
    print(f' \t{np.mean((mondrian["uncal"][attr] > 0.5) == (mondrian["y"] == 1)): .3f} \t{np.mean((mondrian_x["uncal"][attr] > 0.5) == (mondrian_x["y"] == 1)): .3f} \t{np.mean((model_split["uncal"][attr] > 0.5) == (model_split["y"][attr] == 1)): .3f}')
print()
for attr in attribute_names:
    print(f'{attr} LogLoss: \t{log_loss(mondrian["y"], mondrian["proba"][attr]): .3f} \t{log_loss(mondrian_x["y"], mondrian_x["proba"][attr]): .3f} \t{log_loss(model_split["y"][attr], model_split["proba"][attr]): .3f}', end='')
    print(f' \t{log_loss(mondrian["y"], mondrian["uncal"][attr]): .3f} \t{log_loss(mondrian_x["y"], mondrian_x["uncal"][attr]): .3f} \t{log_loss(model_split["y"][attr], model_split["uncal"][attr]): .3f}')
    

	 CondIncl	 CondExcl	 ModelSplit	 CondIncl	 CondExcl	 ModelSplit
Category	 VA	 VA	 VA	 Uncal	 Uncal	 Uncal
base Accuracy: 	 0.8553 	 0.8330 	 0.8558 	 0.8558 	 0.8326 	 0.8559
rand Accuracy: 	 0.8544 	 0.8324 	 0.8497 	 0.8558 	 0.8326 	 0.8491
race Accuracy: 	 0.8548 	 0.8329 	 0.8530 	 0.8558 	 0.8326 	 0.8530
sex Accuracy: 	 0.8555 	 0.8318 	 0.8556 	 0.8558 	 0.8326 	 0.8552
edu Accuracy: 	 0.8561 	 0.8344 	 0.8520 	 0.8558 	 0.8326 	 0.8502

base LogLoss: 	 0.3175 	 0.3588 	 0.3169 	 0.3674 	 0.4111 	 0.3596
rand LogLoss: 	 0.3197 	 0.3604 	 0.3286 	 0.3674 	 0.4111 	 0.3570
race LogLoss: 	 0.3179 	 0.3587 	 0.3217 	 0.3674 	 0.4111 	 0.3641
sex LogLoss: 	 0.3166 	 0.3585 	 0.3197 	 0.3674 	 0.4111 	 0.3713
edu LogLoss: 	 0.3189 	 0.3587 	 0.3284 	 0.3674 	 0.4111 	 0.3916


In [276]:
import pickle

with open(f'../evaluation/results_mondrian_{dataset}_{str(num_rep)}.pkl', 'wb') as file:
    pickle.dump(results, file)


# German


In [277]:
dataset = 'German'
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
statlog_german_credit_data = fetch_ucirepo(id=144) 
  
# data (as pandas dataframes) 
X = statlog_german_credit_data.data.features 
y = statlog_german_credit_data.data.targets 

In [278]:
feature_names = statlog_german_credit_data.variables.description[:-1]
feature_names[0] = 'Status'
feature_names[7] = 'Installment rate'
feature_names[15] = 'Existing credits'
feature_names[17] = 'Num people liable'
target_labels = {1: 'Good', 0: 'Bad'}

In [279]:
print(feature_names)
age = 12
sex = 8

0                         Status
1                       Duration
2                 Credit history
3                        Purpose
4                  Credit amount
5          Savings account/bonds
6       Present employment since
7               Installment rate
8        Personal status and sex
9     Other debtors / guarantors
10       Present residence since
11                      Property
12                           Age
13       Other installment plans
14                       Housing
15              Existing credits
16                           Job
17             Num people liable
18                     Telephone
19                foreign worker
Name: description, dtype: object


In [280]:
df = X
target = 'Class'
df[target] = y
df = df.dropna()
df, categorical_features, categorical_labels, _, _ = transform_to_numeric(df, target)

In [281]:
num_to_test = 10 # number of instances to test, one from each class

df = df.sample(frac=1, random_state=42).sort_values(by=[target])
Xd, yd = df.drop(target,axis=1), df[target] 
no_of_classes = len(np.unique(yd))
no_of_features = Xd.shape[1]
no_of_instances = Xd.shape[0]

# select test instances from each class and split into train, cal and test
X, y = Xd.values, yd.values
y[y == 2] = 0

In [282]:
# Mondrian with feature exclusion

base = 'base'
rand = 'rand'
attributes = {base:base, rand:rand, 'sex':sex, 'age':age}
attribute_names = [base, rand, 'sex', 'age']
va = {}
mondrian_x = {}
mondrian_x['uncal'] = {}
mondrian_x['proba'] = {}
mondrian_x['low'] = {}
mondrian_x['high'] = {}
mondrian_x['width'] = {}
mondrian_x['y'] = []
time_mondrian_x = time()
for attr in attribute_names:
    va[attr] = None
    mondrian_x['uncal'][attr] = []
    mondrian_x['proba'][attr] = []
    mondrian_x['low'][attr] = []
    mondrian_x['high'][attr] = []
    mondrian_x['width'][attr] = []

for l in range(num_rep):
    np.random.seed(l)
    indices = np.random.permutation(no_of_instances)
    X = X[indices,:]
    y = y[indices]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        x_train, x_cal, x_test = X_train, X_cal, X_test        
        for attr, a in attributes.items():
            if attr not in [base, rand]:
                x_train = np.delete(x_train, a, axis=1)
                x_cal = np.delete(x_cal, a, axis=1)
                x_test = np.delete(x_test, a, axis=1)
        model = RandomForestClassifier()
        model.fit(x_train,y_train)
        
        age_cal_bin, age_cal_boundaries = binning(X_cal[:,age], bins=num_bins)
        age_test_bin = binning(X_test[:,age], age_cal_boundaries)
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=num_bins)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        cal_bins = {base: None, rand: rand_cal_bin, 'sex': X_cal[:,sex], 'age': age_cal_bin}
        test_bins = {base: None, rand: rand_test_bin, 'sex': X_test[:,sex], 'age': age_test_bin}
        
        mondrian_x['y'].append(y_test)
        
        for attr, a in attributes.items():
            uncal = model.predict_proba(x_test)
            va[attr] = VennAbers(model.predict_proba(x_cal), y_cal, model, bins=cal_bins[attr])
            proba, low, high = va[attr].predict_proba(x_test, output_interval=True, bins=test_bins[attr])
            mondrian_x['uncal'][attr].append(uncal[:,1])
            mondrian_x['proba'][attr].append(proba[:,1])
            mondrian_x['low'][attr].append(low)
            mondrian_x['high'][attr].append(high)
            mondrian_x['width'][attr].append(high-low)

results[dataset]['mondrian_x']['time'] = time() - time_mondrian_x
results[dataset]['mondrian_x']['struct'] = mondrian_x

In [283]:
# Mondrian with all features

base = 'base'
rand = 'rand'
attributes = [base, rand, sex, age]
attribute_names = [base, rand, 'sex', 'age']
va = {}
mondrian = {}
mondrian['uncal'] = {}
mondrian['proba'] = {}
mondrian['low'] = {}
mondrian['high'] = {}
mondrian['width'] = {}
mondrian['y'] = []
time_mondrian = time()
for attr in attribute_names:
    va[attr] = None
    mondrian['uncal'][attr] = []
    mondrian['proba'][attr] = []
    mondrian['low'][attr] = []
    mondrian['high'][attr] = []
    mondrian['width'][attr] = []

time_mondrian = time()
for l in range(num_rep):
    np.random.seed(l)
    indices = np.random.permutation(no_of_instances)
    X = X[indices,:]
    y = y[indices]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        
        model = RandomForestClassifier()
        model.fit(X_train,y_train)
        
        age_cal_bin, age_cal_boundaries = binning(X_cal[:,age], bins=num_bins)
        age_test_bin = binning(X_test[:,age], age_cal_boundaries)
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=num_bins)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        cal_bins = {base: None, rand: rand_cal_bin, 'sex': X_cal[:,sex], 'age': age_cal_bin}
        test_bins = {base: None, rand: rand_test_bin, 'sex': X_test[:,sex], 'age': age_test_bin}
        
        mondrian['y'].append(y_test)
        
        for attr in attribute_names:
            uncal = model.predict_proba(X_test)
            va[attr] = VennAbers(model.predict_proba(X_cal), y_cal, model, bins=cal_bins[attr])
            proba, low, high = va[attr].predict_proba(X_test, output_interval=True, bins=test_bins[attr])
            mondrian['uncal'][attr].append(uncal[:,1])
            mondrian['proba'][attr].append(proba[:,1])
            mondrian['low'][attr].append(low)
            mondrian['high'][attr].append(high)
            mondrian['width'][attr].append(high-low)

results[dataset]['mondrian']['time'] = time() - time_mondrian
results[dataset]['mondrian']['struct'] = mondrian

In [284]:
# Model split based on Mondrian categories

base = 'base'
rand = 'rand'
attributes = [base, rand, sex, age]
attribute_names = [base, rand, 'sex', 'age']
model_split = {}
model_split['uncal'] = {}
model_split['proba'] = {}
model_split['low'] = {}
model_split['high'] = {}
model_split['width'] = {}
model_split['y'] = {}
for attr in attribute_names:
    model_split['uncal'][attr] = []
    model_split['proba'][attr] = []
    model_split['low'][attr] = []
    model_split['high'][attr] = []
    model_split['width'][attr] = []
    model_split['y'][attr] = []

time_split = time()
for l in range(num_rep):
    np.random.seed(l)
    indices = np.random.permutation(no_of_instances)
    X = X[indices,:]
    y = y[indices]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        
        age_cal_bin, age_cal_boundaries = binning(X_cal[:,age], bins=num_bins)
        age_train_bin = binning(X_train[:,age], age_cal_boundaries)
        age_test_bin = binning(X_test[:,age], age_cal_boundaries)
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=num_bins)
        rand_train_bin = binning(np.random.rand(len(y_train)), rand_cal_boundaries)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        train_bins = {base: np.ones(len(y_train)), rand: rand_train_bin, 'sex': X_train[:,sex], 'age': age_train_bin}
        cal_bins = {base: np.ones(len(y_cal)), rand: rand_cal_bin, 'sex': X_cal[:,sex], 'age': age_cal_bin}
        test_bins = {base: np.ones(len(y_test)), rand: rand_test_bin, 'sex': X_test[:,sex], 'age': age_test_bin}
        
        
        for attr in attribute_names:
            for bin in np.unique(train_bins[attr]):
                X_train_bin = X_train[train_bins[attr] == bin]
                y_train_bin = y_train[train_bins[attr] == bin]
                X_cal_bin = X_cal[cal_bins[attr] == bin]
                y_cal_bin = y_cal[cal_bins[attr] == bin]
                X_test_bin = X_test[test_bins[attr] == bin]
                y_test_bin = y_test[test_bins[attr] == bin]
                
                if len(y_train_bin) == 0 or len(y_cal_bin) == 0 or len(y_test_bin) == 0:
                    continue
                
                model = RandomForestClassifier()                
                model.fit(X_train_bin,y_train_bin)
                uncal = model.predict_proba(X_test_bin)
                va = VennAbers(model.predict_proba(X_cal_bin), y_cal_bin, model)
                proba, low, high = va.predict_proba(X_test_bin, output_interval=True)
                model_split['uncal'][attr].append(uncal[:,1])
                model_split['proba'][attr].append(proba[:,1])
                model_split['low'][attr].append(low)
                model_split['high'][attr].append(high)
                model_split['width'][attr].append(high - low)
                model_split['y'][attr].append(y_test_bin)

results[dataset]['model_split']['time'] = time() - time_split
results[dataset]['model_split']['struct'] = model_split

In [285]:
for attr in attribute_names:
    mondrian_x['uncal'][attr] = np.concatenate(mondrian_x['uncal'][attr])
    mondrian_x['proba'][attr] = np.concatenate(mondrian_x['proba'][attr]) 
    mondrian_x['low'][attr] = np.concatenate(mondrian_x['low'][attr]) 
    mondrian_x['high'][attr] = np.concatenate(mondrian_x['high'][attr]) 
    mondrian_x['width'][attr] = np.concatenate(mondrian_x['width'][attr]) 
    mondrian['uncal'][attr] = np.concatenate(mondrian['uncal'][attr])
    mondrian['proba'][attr] = np.concatenate(mondrian['proba'][attr]) 
    mondrian['low'][attr] = np.concatenate(mondrian['low'][attr]) 
    mondrian['high'][attr] = np.concatenate(mondrian['high'][attr]) 
    mondrian['width'][attr] = np.concatenate(mondrian['width'][attr]) 
    model_split['uncal'][attr] = np.concatenate(model_split['uncal'][attr])
    model_split['proba'][attr] = np.concatenate(model_split['proba'][attr]) 
    model_split['low'][attr] = np.concatenate(model_split['low'][attr]) 
    model_split['high'][attr] = np.concatenate(model_split['high'][attr]) 
    model_split['width'][attr] = np.concatenate(model_split['width'][attr]) 
    model_split['y'][attr] = np.concatenate(model_split['y'][attr]) 

mondrian_x['y'] = np.concatenate(mondrian_x['y']) 
mondrian['y'] = np.concatenate(mondrian['y']) 

In [286]:
print('Category\t CondIncl\t CondExcl\t ModelSplit')
for attr in attribute_names:
    results[dataset]['mondrian_x'][attr] = np.mean(mondrian_x['width'][attr])
    results[dataset]['mondrian'][attr] = np.mean(mondrian['width'][attr])
    results[dataset]['model_split'][attr] = np.mean(model_split['width'][attr])
    print(f'{attr} width: \t{np.mean(mondrian["width"][attr]): .3f} \t{np.mean(mondrian_x["width"][attr]): .3f} \t{np.mean(model_split["width"][attr]): .3f}')

Category	 CondIncl	 CondExcl	 ModelSplit
base width: 	 0.0429 	 0.0407 	 0.0450
rand width: 	 0.1227 	 0.1232 	 0.1235
sex width: 	 0.0964 	 0.0909 	 0.0943
age width: 	 0.1228 	 0.1187 	 0.1130


In [287]:
print('\t CondIncl\t CondExcl\t ModelSplit\t CondIncl\t CondExcl\t ModelSplit')
print('Category\t VA\t VA\t VA\t Uncal\t Uncal\t Uncal')
for attr in attribute_names:
    print(f'{attr} Accuracy: \t{np.mean((mondrian["proba"][attr] > 0.5) == (mondrian["y"] == 1)): .3f} \t{np.mean((mondrian_x["proba"][attr] > 0.5) == (mondrian_x["y"] == 1)): .3f} \t{np.mean((model_split["proba"][attr] > 0.5) == (model_split["y"][attr] == 1)): .3f}', end='')
    print(f' \t{np.mean((mondrian["uncal"][attr] > 0.5) == (mondrian["y"] == 1)): .3f} \t{np.mean((mondrian_x["uncal"][attr] > 0.5) == (mondrian_x["y"] == 1)): .3f} \t{np.mean((model_split["uncal"][attr] > 0.5) == (model_split["y"][attr] == 1)): .3f}')
print()
for attr in attribute_names:
    print(f'{attr} LogLoss: \t{log_loss(mondrian["y"], mondrian["proba"][attr]): .3f} \t{log_loss(mondrian_x["y"], mondrian_x["proba"][attr]): .3f} \t{log_loss(model_split["y"][attr], model_split["proba"][attr]): .3f}', end='')
    print(f' \t{log_loss(mondrian["y"], mondrian["uncal"][attr]): .3f} \t{log_loss(mondrian_x["y"], mondrian_x["uncal"][attr]): .3f} \t{log_loss(model_split["y"][attr], model_split["uncal"][attr]): .3f}')
    

	 CondIncl	 CondExcl	 ModelSplit	 CondIncl	 CondExcl	 ModelSplit
Category	 VA	 VA	 VA	 Uncal	 Uncal	 Uncal
base Accuracy: 	 0.7710 	 0.7540 	 0.7530 	 0.7710 	 0.7520 	 0.7580
rand Accuracy: 	 0.7600 	 0.7450 	 0.7250 	 0.7710 	 0.7520 	 0.7230
sex Accuracy: 	 0.7440 	 0.7440 	 0.7260 	 0.7710 	 0.7520 	 0.7300
age Accuracy: 	 0.7610 	 0.7460 	 0.7310 	 0.7710 	 0.7520 	 0.7390

base LogLoss: 	 0.5036 	 0.5143 	 0.5090 	 0.5009 	 0.5125 	 0.5046
rand LogLoss: 	 0.5070 	 0.5382 	 0.5448 	 0.5009 	 0.5125 	 0.5369
sex LogLoss: 	 0.5105 	 0.5167 	 0.5396 	 0.5009 	 0.5125 	 0.5597
age LogLoss: 	 0.5108 	 0.5211 	 0.5539 	 0.5009 	 0.5125 	 0.5298


In [288]:
import pickle

with open(f'../evaluation/results_mondrian_{dataset}_{str(num_rep)}.pkl', 'wb') as file:
    pickle.dump(results, file)


# Boston

In [6]:
dataset = 'Housing'
df = pd.read_csv('../data/reg/HousingData.csv', na_values='NA')
# df.head()

In [7]:
feature_names = df.columns[:-1]
print(feature_names)
crm = 0

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT'],
      dtype='object')


In [8]:
target = 'MEDV'
df = df.dropna()
df, categorical_features, categorical_labels, target_labels, _ = transform_to_numeric(df, target)

In [9]:
num_to_test = 10 # number of instances to test, one from each class

Xd, yd = df.drop(target,axis=1), df[target]
X, y = df.drop(target,axis=1).values, df[target].values
no_of_classes = len(np.unique(yd))
no_of_features = Xd.shape[1]
no_of_instances = Xd.shape[0]

### Standard Regression
Default confidence of 90%

In [17]:
# Mondrian with feature exclusion

base = 'base'
rand = 'rand'
attributes = {base:base, rand:rand, 'crm':crm}
attribute_names = [base, rand, 'crm']
cps = {}
mondrian_x = {}
mondrian_x['uncal'] = {}
mondrian_x['median'] = {}
mondrian_x['low'] = {}
mondrian_x['high'] = {}
mondrian_x['width'] = {}
mondrian_x['y'] = []
for attr in attribute_names:
    cps[attr] = None
    mondrian_x['uncal'][attr] = []
    mondrian_x['median'][attr] = []
    mondrian_x['low'][attr] = []
    mondrian_x['high'][attr] = []
    mondrian_x['width'][attr] = []

time_mondrian = time()
for l in range(num_rep):
    np.random.seed(l)
    indices = np.random.permutation(no_of_instances)
    X = X[indices,:]
    y = y[indices]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        x_train, x_cal, x_test = X_train, X_cal, X_test        
        for attr, a in attributes.items():
            if attr not in [base, rand]:
                x_train = np.delete(x_train, a, axis=1)
                x_cal = np.delete(x_cal, a, axis=1)
                x_test = np.delete(x_test, a, axis=1)
        model = RandomForestRegressor()
        model.fit(x_train,y_train)
        
        crm_cal_bin, crm_cal_boundaries = binning(X_cal[:,crm], bins=num_bins)
        crm_test_bin = binning(X_test[:,crm], crm_cal_boundaries)    
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=num_bins)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        cal_bins = {base: None, rand: rand_cal_bin, 'crm': crm_cal_bin}
        test_bins = {base: None, rand: rand_test_bin, 'crm': crm_test_bin}
        residuals_cal = model.predict(x_cal) - y_cal
        y_test_hat = model.predict(x_test)
        mondrian_x['y'].append(y_test)        
        
        for attr, a in attributes.items():                
            cps[attr] = ConformalPredictiveSystem()
            cps[attr].fit(residuals_cal, bins=cal_bins[attr])
            values = cps[attr].predict(y_test_hat, bins=test_bins[attr], lower_percentiles=[5, 50], higher_percentiles=[95, 50])
            mondrian_x['uncal'][attr].append(y_test_hat)
            mondrian_x['median'][attr].append(np.mean(values[:,[1, 3]], axis=1))
            mondrian_x['low'][attr].append(values[:,0])
            mondrian_x['high'][attr].append(values[:,2])
            mondrian_x['width'][attr].append(values[:,2]-values[:,0])

results[dataset]['mondrian_x']['time'] = time() - time_mondrian
results[dataset]['mondrian_x']['struct'] = mondrian_x

In [18]:
# Mondrian with all features

base = 'base'
rand = 'rand'
attributes = [base, rand, crm]
attribute_names = [base, rand, 'crm']
cps = {}
mondrian = {}
mondrian['uncal'] = {}
mondrian['median'] = {}
mondrian['low'] = {}
mondrian['high'] = {}
mondrian['width'] = {}
mondrian['y'] = []
for attr in attribute_names:
    cps[attr] = None
    mondrian['uncal'][attr] = []
    mondrian['median'][attr] = []
    mondrian['low'][attr] = []
    mondrian['high'][attr] = []
    mondrian['width'][attr] = []

time_mondrian = time()
for l in range(num_rep):
    np.random.seed(l)
    indices = np.random.permutation(no_of_instances)
    X = X[indices,:]
    y = y[indices]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        
        model = RandomForestRegressor()
        model.fit(X_train,y_train)
        
        crm_cal_bin, crm_cal_boundaries = binning(X_cal[:,crm], bins=num_bins)
        crm_test_bin = binning(X_test[:,crm], crm_cal_boundaries)    
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=num_bins)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        cal_bins = {base: None, rand: rand_cal_bin, 'crm': crm_cal_bin}
        test_bins = {base: None, rand: rand_test_bin, 'crm': crm_test_bin}
        residuals_cal = model.predict(X_cal) - y_cal
        y_test_hat = model.predict(X_test)
        mondrian['y'].append(y_test)
        
        for attr in attribute_names:
            cps[attr] = ConformalPredictiveSystem()
            cps[attr].fit(residuals_cal, bins=cal_bins[attr])
            values = cps[attr].predict(y_test_hat, bins=test_bins[attr], lower_percentiles=[5, 50], higher_percentiles=[95, 50])
            mondrian['uncal'][attr].append(y_test_hat)
            mondrian['median'][attr].append(np.mean(values[:,[1, 3]], axis=1))
            mondrian['low'][attr].append(values[:,0])
            mondrian['high'][attr].append(values[:,2])
            mondrian['width'][attr].append(values[:,2]-values[:,0])

results[dataset]['mondrian']['time'] = time() - time_mondrian
results[dataset]['mondrian']['struct'] = mondrian

In [19]:
# model split based on Mondrian categories

base = 'base'
rand = 'rand'
attributes = [base, rand, crm]
attribute_names = [base, rand, 'crm']
cps = {}
model_split = {}
model_split['uncal'] = {}
model_split['median'] = {}
model_split['low'] = {}
model_split['high'] = {}
model_split['width'] = {}
model_split['y'] = {}
for attr in attribute_names:
    cps[attr] = None
    model_split['uncal'][attr] = []
    model_split['median'][attr] = []
    model_split['low'][attr] = []
    model_split['high'][attr] = []
    model_split['width'][attr] = []
    model_split['y'][attr] = []

time_split = time()
for l in range(num_rep):
    np.random.seed(l)
    indices = np.random.permutation(no_of_instances)
    X = X[indices,:]
    y = y[indices]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        
        
        crm_cal_bin, crm_cal_boundaries = binning(X_cal[:,crm], bins=num_bins)
        crm_train_bin = binning(X_train[:,crm], crm_cal_boundaries)
        crm_test_bin = binning(X_test[:,crm], crm_cal_boundaries)    
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=num_bins)
        rand_train_bin = binning(np.random.rand(len(y_train)), rand_cal_boundaries)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        train_bins = {base: np.ones(len(y_train)), rand: rand_train_bin, 'crm': crm_train_bin}
        cal_bins = {base: np.ones(len(y_cal)), rand: rand_cal_bin, 'crm': crm_cal_bin}
        test_bins = {base: np.ones(len(y_test)), rand: rand_test_bin, 'crm': crm_test_bin}
        
        
        for attr in attribute_names:
            for bin in np.unique(train_bins[attr]):
                X_train_bin = X_train[train_bins[attr] == bin,:]
                y_train_bin = y_train[train_bins[attr] == bin]
                X_cal_bin = X_cal[cal_bins[attr] == bin,:]
                y_cal_bin = y_cal[cal_bins[attr] == bin]
                X_test_bin = X_test[test_bins[attr] == bin,:]
                y_test_bin = y_test[test_bins[attr] == bin]
                
                if len(y_train_bin) == 0 or len(y_cal_bin) == 0 or len(y_test_bin) == 0:
                    continue
                
                model = RandomForestRegressor()
                model.fit(X_train_bin, y_train_bin)
                
                residuals_cal = model.predict(X_cal_bin) - y_cal_bin
                y_test_bin_hat = model.predict(X_test_bin)
            
                cps[attr] = ConformalPredictiveSystem()
                cps[attr].fit(residuals_cal)
                values = cps[attr].predict(y_test_bin_hat, lower_percentiles=[5, 50], higher_percentiles=[95, 50])
                model_split['uncal'][attr].append(y_test_bin_hat)
                model_split['median'][attr].append(np.mean(values[:,[1, 3]], axis=1))
                model_split['low'][attr].append(values[:,0])
                model_split['high'][attr].append(values[:,2])
                model_split['width'][attr].append(values[:,2]-values[:,0])
                model_split['y'][attr].append(y_test_bin)

results[dataset]['model_split']['time'] = time() - time_split
results[dataset]['model_split']['struct'] = model_split

In [20]:
for attr in attribute_names:
    mondrian_x['uncal'][attr] = np.concatenate(mondrian_x['uncal'][attr])
    mondrian_x['median'][attr] = np.concatenate(mondrian_x['median'][attr]) 
    mondrian_x['low'][attr] = np.concatenate(mondrian_x['low'][attr]) 
    mondrian_x['high'][attr] = np.concatenate(mondrian_x['high'][attr]) 
    mondrian_x['width'][attr] = np.concatenate(mondrian_x['width'][attr]) 
    mondrian['uncal'][attr] = np.concatenate(mondrian['uncal'][attr])
    mondrian['median'][attr] = np.concatenate(mondrian['median'][attr]) 
    mondrian['low'][attr] = np.concatenate(mondrian['low'][attr]) 
    mondrian['high'][attr] = np.concatenate(mondrian['high'][attr]) 
    mondrian['width'][attr] = np.concatenate(mondrian['width'][attr]) 
    model_split['uncal'][attr] = np.concatenate(model_split['uncal'][attr])
    model_split['median'][attr] = np.concatenate(model_split['median'][attr]) 
    model_split['low'][attr] = np.concatenate(model_split['low'][attr]) 
    model_split['high'][attr] = np.concatenate(model_split['high'][attr]) 
    model_split['width'][attr] = np.concatenate(model_split['width'][attr]) 
    model_split['y'][attr] = np.concatenate(model_split['y'][attr]) 

mondrian_x['y'] = np.concatenate(mondrian_x['y']) 
mondrian['y'] = np.concatenate(mondrian['y']) 

In [22]:
print('Category\t CondIncl\t CondExcl\t ModelSplit')
for attr in attribute_names:
    results[dataset]['mondrian_x'][attr] = np.mean(mondrian_x['width'][attr])
    results[dataset]['mondrian'][attr] = np.mean(mondrian['width'][attr])
    results[dataset]['model_split'][attr] = np.mean(model_split['width'][attr])
    print(f'{attr} width: \t{np.mean(mondrian["width"][attr]): .3f} \t{np.mean(mondrian_x["width"][attr]): .3f} \t{np.mean(model_split["width"][attr]): .3f}')

Category	 CondIncl	 CondExcl	 ModelSplit
base width: 	 10.785 	 9.203 	 10.047
rand width: 	 13.852 	 15.709 	 19.781
crm width: 	 13.084 	 13.155 	 17.124


: 

In [21]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score

print('\t CondIncl\t CondExcl\t ModelSplit\t CondIncl\t CondExcl\t ModelSplit')
print('Category\t VA\t VA\t VA\t Uncal\t Uncal\t Uncal')
for attr in attribute_names:
    print(f'{attr} Coverage: \t{coverage(mondrian["y"], mondrian["low"][attr], mondrian["high"][attr]): .3f} \t{coverage(mondrian_x["y"], mondrian_x["low"][attr], mondrian_x["high"][attr]): .3f} \t{coverage(model_split["y"][attr], model_split["low"][attr], model_split["high"][attr]): .3f}')
# for attr in attribute_names:
#     print(f'{attr} MAE: \t{mean_absolute_error(mondrian["y"], mondrian["median"][attr]): .3f} \t{mean_absolute_error(mondrian_x["y"], mondrian_x["median"][attr]): .3f} \t{mean_absolute_error(model_split["y"][attr], model_split["median"][attr]): .3f}', end='')
#     print(f' \t{mean_absolute_error(mondrian["y"], mondrian["uncal"][attr]): .3f} \t{mean_absolute_error(mondrian_x["y"], mondrian_x["uncal"][attr]): .3f} \t{mean_absolute_error(model_split["y"][attr], model_split["uncal"][attr]): .3f}')
for attr in attribute_names:
    print(f'{attr} R2: \t{r2_score(mondrian["y"], mondrian["median"][attr]): .3f} \t{r2_score(mondrian_x["y"], mondrian_x["median"][attr]): .3f} \t{r2_score(model_split["y"][attr], model_split["median"][attr]): .3f}', end='')    
    print(f' \t{r2_score(mondrian["y"], mondrian["uncal"][attr]): .3f} \t{r2_score(mondrian_x["y"], mondrian_x["uncal"][attr]): .3f} \t{r2_score(model_split["y"][attr], model_split["uncal"][attr]): .3f}')

	 CondIncl	 CondExcl	 ModelSplit	 CondIncl	 CondExcl	 ModelSplit
Category	 VA	 VA	 VA	 Uncal	 Uncal	 Uncal
base Coverage: 	 0.929 	 0.878 	 0.909
rand Coverage: 	 0.919 	 0.896 	 0.914
crm Coverage: 	 0.896 	 0.888 	 0.911
base R2: 	 0.857 	 0.841 	 0.855 	 0.859 	 0.842 	 0.862
rand R2: 	 0.855 	 0.833 	 0.692 	 0.859 	 0.842 	 0.710
crm R2: 	 0.856 	 0.829 	 0.816 	 0.859 	 0.842 	 0.829


In [299]:
import pickle

with open(f'../evaluation/results_mondrian_{dataset}_{str(num_rep)}.pkl', 'wb') as file:
    pickle.dump(results, file)


## Communities and Crimes

In [300]:
dataset = 'CaC'
# Load the CSV file
df = pd.read_csv('../data/reg/communities.csv')

# Remove columns with missing values
df = df.dropna(axis=1)

# Remove string columns
df = df.select_dtypes(exclude=['object'])

# df.head()


In [301]:
feature_names = df.columns[:-1]
for i, feature_name in enumerate(feature_names):
    print(f'{i}:{feature_name}', end=", ") 
blk = 4

0:state, 1:fold, 2:population, 3:householdsize, 4:racepctblack, 5:racePctWhite, 6:racePctAsian, 7:racePctHisp, 8:agePct12t21, 9:agePct12t29, 10:agePct16t24, 11:agePct65up, 12:numbUrban, 13:pctUrban, 14:medIncome, 15:pctWWage, 16:pctWFarmSelf, 17:pctWInvInc, 18:pctWSocSec, 19:pctWPubAsst, 20:pctWRetire, 21:medFamInc, 22:perCapInc, 23:whitePerCap, 24:blackPerCap, 25:indianPerCap, 26:AsianPerCap, 27:HispPerCap, 28:NumUnderPov, 29:PctPopUnderPov, 30:PctLess9thGrade, 31:PctNotHSGrad, 32:PctBSorMore, 33:PctUnemployed, 34:PctEmploy, 35:PctEmplManu, 36:PctEmplProfServ, 37:PctOccupManu, 38:PctOccupMgmtProf, 39:MalePctDivorce, 40:MalePctNevMarr, 41:FemalePctDiv, 42:TotalPctDiv, 43:PersPerFam, 44:PctFam2Par, 45:PctKids2Par, 46:PctYoungKids2Par, 47:PctTeen2Par, 48:PctWorkMomYoungKids, 49:PctWorkMom, 50:NumIlleg, 51:PctIlleg, 52:NumImmig, 53:PctImmigRecent, 54:PctImmigRec5, 55:PctImmigRec8, 56:PctImmigRec10, 57:PctRecentImmig, 58:PctRecImmig5, 59:PctRecImmig8, 60:PctRecImmig10, 61:PctSpeakEnglOnly,

In [302]:
target = 'ViolentCrimesPerPop'
df, categorical_features, categorical_labels, target_labels, _ = transform_to_numeric(df, target)

In [303]:
Xd, yd = df.drop(target,axis=1), df[target]
X, y = df.drop(target,axis=1).values, df[target].values
no_of_classes = len(np.unique(yd))
no_of_features = Xd.shape[1]
no_of_instances = Xd.shape[0]

In [304]:
# Mondrian with feature exclusion

base = 'base'
rand = 'rand'
attributes = {base:base, rand:rand, 'race':blk}
attribute_names = [base, rand, 'race']
cps = {}
mondrian_x = {}
mondrian_x['uncal'] = {}
mondrian_x['median'] = {}
mondrian_x['low'] = {}
mondrian_x['high'] = {}
mondrian_x['width'] = {}
mondrian_x['y'] = []
for attr in attribute_names:
    cps[attr] = None
    mondrian_x['uncal'][attr] = []
    mondrian_x['median'][attr] = []
    mondrian_x['low'][attr] = []
    mondrian_x['high'][attr] = []
    mondrian_x['width'][attr] = []

time_mondrian = time()
for l in range(num_rep):
    np.random.seed(l)
    indices = np.random.permutation(no_of_instances)
    X = X[indices,:]
    y = y[indices]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        x_train, x_cal, x_test = X_train, X_cal, X_test        
        for attr, a in attributes.items():
            if attr not in [base, rand]:
                x_train = np.delete(x_train, a, axis=1)
                x_cal = np.delete(x_cal, a, axis=1)
                x_test = np.delete(x_test, a, axis=1)
        model = RandomForestRegressor()
        model.fit(x_train,y_train)
        
        blk_cal_bin, blk_cal_boundaries = binning(X_cal[:,blk], bins=num_bins)
        blk_test_bin = binning(X_test[:,blk], blk_cal_boundaries)    
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=num_bins)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        cal_bins = {base: None, rand: rand_cal_bin, 'race': blk_cal_bin}
        test_bins = {base: None, rand: rand_test_bin, 'race': blk_test_bin}
        residuals_cal = model.predict(x_cal) - y_cal
        y_test_hat = model.predict(x_test)
        mondrian_x['y'].append(y_test)        
        
        for attr, a in attributes.items():                
            cps[attr] = ConformalPredictiveSystem()
            cps[attr].fit(residuals_cal, bins=cal_bins[attr])
            values = cps[attr].predict(y_test_hat, bins=test_bins[attr], lower_percentiles=[5, 50], higher_percentiles=[95, 50])
            mondrian_x['uncal'][attr].append(y_test_hat)
            mondrian_x['median'][attr].append(np.mean(values[:,[1, 3]], axis=1))
            mondrian_x['low'][attr].append(values[:,0])
            mondrian_x['high'][attr].append(values[:,2])
            mondrian_x['width'][attr].append(values[:,2]-values[:,0])

results[dataset]['mondrian_x']['time'] = time() - time_mondrian
results[dataset]['mondrian_x']['struct'] = mondrian_x

In [305]:
# Mondrian with all features

base = 'base'
rand = 'rand'
attributes = [base, rand, blk]
attribute_names = [base, rand, 'race']
cps = {}
mondrian = {}
mondrian['uncal'] = {}
mondrian['median'] = {}
mondrian['low'] = {}
mondrian['high'] = {}
mondrian['width'] = {}
mondrian['y'] = []
for attr in attribute_names:
    cps[attr] = None
    mondrian['uncal'][attr] = []
    mondrian['median'][attr] = []
    mondrian['low'][attr] = []
    mondrian['high'][attr] = []
    mondrian['width'][attr] = []

time_mondrian = time()
for l in range(num_rep):
    np.random.seed(l)
    indices = np.random.permutation(no_of_instances)
    X = X[indices,:]
    y = y[indices]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        
        model = RandomForestRegressor()
        model.fit(X_train,y_train)

        blk_cal_bin, blk_cal_boundaries = binning(X_cal[:,blk], bins=num_bins)
        blk_test_bin = binning(X_test[:,blk], blk_cal_boundaries)    
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=num_bins)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        cal_bins = {base: None, rand: rand_cal_bin, 'race': blk_cal_bin}
        test_bins = {base: None, rand: rand_test_bin, 'race': blk_test_bin}
        residuals_cal = model.predict(X_cal) - y_cal
        y_test_hat = model.predict(X_test)
        mondrian['y'].append(y_test)
        
        for attr in attribute_names:
            cps[attr] = ConformalPredictiveSystem()
            cps[attr].fit(residuals_cal, bins=cal_bins[attr])
            values = cps[attr].predict(y_test_hat, bins=test_bins[attr], lower_percentiles=[5, 50], higher_percentiles=[95, 50])
            mondrian['uncal'][attr].append(y_test_hat)
            mondrian['median'][attr].append(np.mean(values[:,[1, 3]], axis=1))
            mondrian['low'][attr].append(values[:,0])
            mondrian['high'][attr].append(values[:,2])
            mondrian['width'][attr].append(values[:,2]-values[:,0])

results[dataset]['mondrian']['time'] = time() - time_mondrian
results[dataset]['mondrian']['struct'] = mondrian

In [306]:
# model split based on Mondrian categories

base = 'base'
rand = 'rand'
attributes = [base, rand, blk]
attribute_names = [base, rand, 'race']
cps = {}
model_split = {}
model_split['uncal'] = {}
model_split['median'] = {}
model_split['low'] = {}
model_split['high'] = {}
model_split['width'] = {}
model_split['y'] = {}
for attr in attribute_names:
    cps[attr] = None
    model_split['uncal'][attr] = []
    model_split['median'][attr] = []
    model_split['low'][attr] = []
    model_split['high'][attr] = []
    model_split['width'][attr] = []
    model_split['y'][attr] = []

time_split = time()
for l in range(num_rep):
    np.random.seed(l)
    indices = np.random.permutation(no_of_instances)
    X = X[indices,:]
    y = y[indices]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        
        
        blk_cal_bin, blk_cal_boundaries = binning(X_cal[:,blk], bins=num_bins)
        blk_train_bin = binning(X_train[:,blk], blk_cal_boundaries)  
        blk_test_bin = binning(X_test[:,blk], blk_cal_boundaries)  
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=num_bins)
        rand_train_bin = binning(np.random.rand(len(y_train)), rand_cal_boundaries)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        train_bins = {base: np.ones(len(y_train)), rand: rand_train_bin, 'race': blk_train_bin}
        cal_bins = {base: np.ones(len(y_cal)), rand: rand_cal_bin, 'race': blk_cal_bin}
        test_bins = {base: np.ones(len(y_test)), rand: rand_test_bin, 'race': blk_test_bin}
        
        
        for attr in attribute_names:
            for bin in np.unique(train_bins[attr]):
                X_train_bin = X_train[train_bins[attr] == bin,:]
                y_train_bin = y_train[train_bins[attr] == bin]
                X_cal_bin = X_cal[cal_bins[attr] == bin,:]
                y_cal_bin = y_cal[cal_bins[attr] == bin]
                X_test_bin = X_test[test_bins[attr] == bin,:]
                y_test_bin = y_test[test_bins[attr] == bin]
                
                if len(y_train_bin) == 0 or len(y_cal_bin) == 0 or len(y_test_bin) == 0:
                    continue
                
                model = RandomForestRegressor()

                model.fit(X_train_bin, y_train_bin)
                
                residuals_cal = model.predict(X_cal_bin) - y_cal_bin
                y_test_bin_hat = model.predict(X_test_bin)
            
                cps[attr] = ConformalPredictiveSystem()
                cps[attr].fit(residuals_cal)
                values = cps[attr].predict(y_test_bin_hat, lower_percentiles=[5, 50], higher_percentiles=[95, 50])
                model_split['uncal'][attr].append(y_test_bin_hat)
                model_split['median'][attr].append(np.mean(values[:,[1, 3]], axis=1))
                model_split['low'][attr].append(values[:,0])
                model_split['high'][attr].append(values[:,2])
                model_split['width'][attr].append(values[:,2]-values[:,0])
                model_split['y'][attr].append(y_test_bin)

results[dataset]['model_split']['time'] = time() - time_split
results[dataset]['model_split']['struct'] = model_split

In [307]:
for attr in attribute_names:
    mondrian_x['uncal'][attr] = np.concatenate(mondrian_x['uncal'][attr])
    mondrian_x['median'][attr] = np.concatenate(mondrian_x['median'][attr]) 
    mondrian_x['low'][attr] = np.concatenate(mondrian_x['low'][attr]) 
    mondrian_x['high'][attr] = np.concatenate(mondrian_x['high'][attr]) 
    mondrian_x['width'][attr] = np.concatenate(mondrian_x['width'][attr]) 
    mondrian['uncal'][attr] = np.concatenate(mondrian['uncal'][attr])
    mondrian['median'][attr] = np.concatenate(mondrian['median'][attr]) 
    mondrian['low'][attr] = np.concatenate(mondrian['low'][attr]) 
    mondrian['high'][attr] = np.concatenate(mondrian['high'][attr]) 
    mondrian['width'][attr] = np.concatenate(mondrian['width'][attr]) 
    model_split['uncal'][attr] = np.concatenate(model_split['uncal'][attr])
    model_split['median'][attr] = np.concatenate(model_split['median'][attr]) 
    model_split['low'][attr] = np.concatenate(model_split['low'][attr]) 
    model_split['high'][attr] = np.concatenate(model_split['high'][attr]) 
    model_split['width'][attr] = np.concatenate(model_split['width'][attr]) 
    model_split['y'][attr] = np.concatenate(model_split['y'][attr]) 

mondrian_x['y'] = np.concatenate(mondrian_x['y']) 
mondrian['y'] = np.concatenate(mondrian['y']) 

In [308]:
print('Category\t CondIncl\t CondExcl\t ModelSplit')
for attr in attribute_names:
    results[dataset]['mondrian_x'][attr] = np.mean(mondrian_x['width'][attr])
    results[dataset]['mondrian'][attr] = np.mean(mondrian['width'][attr])
    results[dataset]['model_split'][attr] = np.mean(model_split['width'][attr])
    print(f'{attr} width: \t{np.mean(mondrian["width"][attr]): .3f} \t{np.mean(mondrian_x["width"][attr]): .3f} \t{np.mean(model_split["width"][attr]): .3f}')

Category	 CondIncl	 CondExcl	 ModelSplit
base width: 	 0.4536 	 0.4654 	 0.4662
rand width: 	 0.4936 	 0.4938 	 0.5273
race width: 	 0.4630 	 0.4358 	 0.4591


In [309]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score

print('\t CondIncl\t CondExcl\t ModelSplit\t CondIncl\t CondExcl\t ModelSplit')
print('Category\t VA\t VA\t VA\t Uncal\t Uncal\t Uncal')
for attr in attribute_names:
    print(f'{attr} Coverage: \t{coverage(mondrian["y"], mondrian["low"][attr], mondrian["high"][attr]): .3f} \t{coverage(mondrian_x["y"], mondrian_x["low"][attr], mondrian_x["high"][attr]): .3f} \t{coverage(model_split["y"][attr], model_split["low"][attr], model_split["high"][attr]): .3f}')
# for attr in attribute_names:
#     print(f'{attr} MAE: \t{mean_absolute_error(mondrian["y"], mondrian["median"][attr]): .3f} \t{mean_absolute_error(mondrian_x["y"], mondrian_x["median"][attr]): .3f} \t{mean_absolute_error(model_split["y"][attr], model_split["median"][attr]): .3f}', end='')
#     print(f' \t{mean_absolute_error(mondrian["y"], mondrian["uncal"][attr]): .3f} \t{mean_absolute_error(mondrian_x["y"], mondrian_x["uncal"][attr]): .3f} \t{mean_absolute_error(model_split["y"][attr], model_split["uncal"][attr]): .3f}')
for attr in attribute_names:
    print(f'{attr} R2: \t{r2_score(mondrian["y"], mondrian["median"][attr]): .3f} \t{r2_score(mondrian_x["y"], mondrian_x["median"][attr]): .3f} \t{r2_score(model_split["y"][attr], model_split["median"][attr]): .3f}', end='')    
    print(f' \t{r2_score(mondrian["y"], mondrian["uncal"][attr]): .3f} \t{r2_score(mondrian_x["y"], mondrian_x["uncal"][attr]): .3f} \t{r2_score(model_split["y"][attr], model_split["uncal"][attr]): .3f}')

	 CondIncl	 CondExcl	 ModelSplit	 CondIncl	 CondExcl	 ModelSplit
Category	 VA	 VA	 VA	 Uncal	 Uncal	 Uncal
base Coverage: 	 0.8912 	 0.9017 	 0.9047
rand Coverage: 	 0.8987 	 0.9062 	 0.9032
race Coverage: 	 0.9203 	 0.8811 	 0.9062
base R2: 	 0.6320 	 0.6171 	 0.6325 	 0.6429 	 0.6376 	 0.6452
rand R2: 	 0.6307 	 0.6154 	 0.5938 	 0.6429 	 0.6376 	 0.6094
race R2: 	 0.6271 	 0.6142 	 0.6045 	 0.6429 	 0.6376 	 0.6284


In [310]:
import pickle

with open(f'../evaluation/results_mondrian_{dataset}_{str(num_rep)}.pkl', 'wb') as file:
    pickle.dump(results, file)


## Summary
### Time

In [311]:
categories = ['mondrian','mondrian_x','model_split',]
print('Dataset\t CondIncl\t CondExcl\t ModelSplit\tType')
for name, type in datasets.items():
    print(f'{name}\t', end='')
    for category in categories:
        print(f'{results[name][category]["time"]: 0.2f}\t\t', end='')
    print(f'{type}')

Dataset	 CondIncl	 CondExcl	 ModelSplit	Type
COMPAS	 16.02		 13.03		 90.55		classification
Adult	 64.59		 71.45		 295.07		classification
German	 4.41		 4.86		 36.44		classification
Housing	 5.29		 5.33		 27.29		regression
CaC	 133.45		 138.94		 383.80		regression


### Width
#### Mondrian vs model split

In [318]:
heading = {'mondrian':'CondIncl', 'mondrian_x':'CondExcl', 'model_split':'ModelSplit'}
for name, type in datasets.items():
    print(name, end='\t')
    for category in categories:
        print(f' {heading[category]}', end='\t')
    print('')
    for attr in results[name]['mondrian'].keys():
        if attr in ['time', 'struct']:
            continue
        if results[name][category][attr] > 0:
            print(attr, end='\t')
            for category in categories:
                val = results[name][category][attr]
                print(f'{val: 0.3f}', end='\t\t')       
            print('')
    print('')

COMPAS	 CondIncl	 CondExcl	 ModelSplit	
base	 0.008		 0.008		 0.007		
rand	 0.023		 0.023		 0.021		
race	 0.018		 0.020		 0.018		
sex	 0.012		 0.013		 0.011		
age	 0.022		 0.025		 0.021		

Adult	 CondIncl	 CondExcl	 ModelSplit	
base	 0.003		 0.003		 0.003		
rand	 0.010		 0.010		 0.010		
race	 0.007		 0.007		 0.007		
sex	 0.005		 0.005		 0.005		
edu	 0.016		 0.015		 0.015		

German	 CondIncl	 CondExcl	 ModelSplit	
base	 0.043		 0.041		 0.045		
rand	 0.123		 0.123		 0.123		
sex	 0.096		 0.091		 0.094		
age	 0.123		 0.119		 0.113		

Housing	 CondIncl	 CondExcl	 ModelSplit	
base	 8.499		 9.810		 9.780		
rand	 11.790		 14.632		 18.927		
crm	 11.776		 14.733		 15.895		

CaC	 CondIncl	 CondExcl	 ModelSplit	
base	 0.454		 0.465		 0.466		
rand	 0.494		 0.494		 0.527		
race	 0.463		 0.436		 0.459		



#### Mondrian

In [313]:
print('', end='\t')
for attr in results[name]['mondrian'].keys():
    if attr in ['time', 'struct']:
        continue
    for category in ['mondrian']:
        print(f' {category}', end='\t')
print('\nDataset', end='\t')
for attr in results[name]['mondrian'].keys():
    if attr in ['time', 'struct']:
        continue
    for category in ['mondrian']:
        print(f' {attr}', end='\t\t')
print('Type')
for name, type in datasets.items():
    print(f'{name}\t', end='')
    for attr in results[name]['mondrian'].keys():
        if attr in ['time', 'struct']:
            continue
        for category in ['mondrian']:
            val = results[name][category][attr]
            print(f'{val: 0.3f}', end='\t\t') if val > 0 else print(' -', end='\t\t')
    print(f'{type}')

	 mondrian	 mondrian	 mondrian	 mondrian	 mondrian	 mondrian	 mondrian	
Dataset	 base		 rand		 race		 sex		 age		 edu		 crm		Type
COMPAS	 0.008		 0.023		 0.018		 0.012		 0.022		 -		 -		classification
Adult	 0.003		 0.010		 0.007		 0.005		 -		 0.016		 -		classification
German	 0.043		 0.123		 -		 0.096		 0.123		 -		 -		classification
Housing	 8.499		 11.790		 -		 -		 -		 -		 11.776		regression
CaC	 0.454		 0.494		 0.463		 -		 -		 -		 -		regression


#### Mondrian With Excluded Features

In [314]:
print('', end='\t')
for attr in results[name]['mondrian_x'].keys():
    if attr in ['time', 'struct']:
        continue
    for category in ['mondrian_x']:
        print(f' {category}', end='\t')
print('\nDataset', end='\t')
for attr in results[name]['mondrian_x'].keys():
    if attr in ['time', 'struct']:
        continue
    for category in ['mondrian_x']:
        print(f' {attr}', end='\t\t')
print('Type')
for name, type in datasets.items():
    print(f'{name}\t', end='')
    for attr in results[name]['mondrian_x'].keys():
        if attr in ['time', 'struct']:
            continue
        for category in ['mondrian_x']:
            val = results[name][category][attr]
            print(f'{val: 0.3f}', end='\t\t') if val > 0 else print(' -', end='\t\t')
    print(f'{type}')

	 mondrian_x	 mondrian_x	 mondrian_x	 mondrian_x	 mondrian_x	 mondrian_x	 mondrian_x	
Dataset	 base		 rand		 race		 sex		 age		 edu		 crm		Type
COMPAS	 0.008		 0.023		 0.020		 0.013		 0.025		 -		 -		classification
Adult	 0.003		 0.010		 0.007		 0.005		 -		 0.015		 -		classification
German	 0.041		 0.123		 -		 0.091		 0.119		 -		 -		classification
Housing	 9.810		 14.632		 -		 -		 -		 -		 14.733		regression
CaC	 0.465		 0.494		 0.436		 -		 -		 -		 -		regression


#### Model Select

In [315]:
print('', end='\t')
for attr in results[name]['model_split'].keys():
    if attr in ['time', 'struct']:
        continue
    for category in ['model_split']:
        print(f' {category}', end='\t')
print('\nDataset', end='\t')
for attr in results[name]['model_split'].keys():
    if attr in ['time', 'struct']:
        continue
    for category in ['model_split']:
        print(f' {attr}', end='\t\t')
print('Type')
for name, type in datasets.items():
    print(f'{name}\t', end='')
    for attr in results[name]['model_split'].keys():
        if attr in ['time', 'struct']:
            continue
        for category in ['model_split']:
            val = results[name][category][attr]
            print(f'{val: 0.3f}', end='\t\t') if val > 0 else print(' -', end='\t\t')
    print(f'{type}')

	 model_split	 model_split	 model_split	 model_split	 model_split	 model_split	 model_split	
Dataset	 base		 rand		 race		 sex		 age		 edu		 crm		Type
COMPAS	 0.007		 0.021		 0.018		 0.011		 0.021		 -		 -		classification
Adult	 0.003		 0.010		 0.007		 0.005		 -		 0.015		 -		classification
German	 0.045		 0.123		 -		 0.094		 0.113		 -		 -		classification
Housing	 9.780		 18.927		 -		 -		 -		 -		 15.895		regression
CaC	 0.466		 0.527		 0.459		 -		 -		 -		 -		regression


In [316]:
import pickle

with open(f'../evaluation/results_mondrian_{str(num_rep)}.pkl', 'wb') as file:
    pickle.dump(results, file)
